### 测试线条提取和转换

1. 标注线延法线取n个点 拿到mask (暂时测试，后续通过分割模型直接拿到mask)
2. 采样延长测试

In [42]:
def sample_normal_line(start_point, normal_vec, num_samples = 10, max_distance = 10 ):
    sampled_points = []
    step_distance = max_distance / num_samples
    for i in range(1, num_samples + 1):
        distance = i * step_distance
        sample_point = start_point + distance * normal_vec
        sampled_points.append(sample_point)
    return sampled_points

In [45]:
from svgpathtools import parse_path, Line, wsvg
import numpy as np
width, height = 1361, 1367
path_d = "M416.5 654C425.999 687.619 430.439 702.048 438 726"
path = parse_path(path_d)
n_points = 500  # 采样点数
points = [path.point(i/n_points) for i in range(n_points+1)]
normals = [path.normal(i/n_points) for i in range(n_points+1)]
normal_lines = []
normal_lines_pixels = []
for point, normal_vec in zip(points, normals):
    normal_line = Line(point, point + normal_vec * 10)
    normal_lines_pixels.extend(sample_normal_line(point, normal_vec))
    normal_lines.append(normal_line)
    # print(f"在参数 T={T:.2f} 处: 点={point}, 法线={normal_vec}")

wsvg(paths=[path] + normal_lines, filename='path_with_normals.svg')

# # 根据图像比例缩放并四舍五入
points_pixels = [(int(round(p.real)), int(round(p.imag))) for p in points]
complete_pixels = [(int(round(p.real)), int(round(p.imag))) for p in [*points, *normal_lines_pixels]]
# len(normal_lines_pixels) # 500个采样点 * 10法线采样点  = 50000
len(complete_pixels) # 500

5511

In [46]:
from PIL import Image
import matplotlib.pyplot as plt


origin = "mask.png"
def extract_pixels_from_origin(pixels): #从原始图像提取像素
    origin_img = Image.open(origin).convert('RGBA')   # 换成你的图片路径
    origin_img = np.array(origin_img)
    new_mask = Image.new('RGBA', (width, height), (255, 255, 255, 0))
    # 填充线条上的像素为黑色
    for x, y in pixels:
        new_mask.putpixel((x, y), tuple(origin_img[y, x]))  # 0 为黑色
    new_mask.save("new_mask.png")

# extract_pixels_from_origin(points_pixels)
extract_pixels_from_origin(complete_pixels)